In [1]:
# Donchian Weekly Classic Trend Following System
import gta_prices
import numpy as np
import pandas as pd

In [2]:
# Trade parameters.
exchange = 'LSE'
tidm = 'CHG'
timeframe = 'Weekly'
filename = f'{exchange}_{tidm}_prices.csv'
p1 = 48 # System 1 look back period.
p2 = 24 # System 2 look back period.
p3 = 12 # System 3 look back period.
p4 = 6  # System 4 look back period.
position_size = 7500  # Position size in major currency unit.
risk_pct = 0.2 # Percentage risk per trade.
commission = 11.95  # Commission per trade.
sduty = 0.5  # Stamp Duty percentage.

In [3]:
# Function definitions.
def charges(date, commission):
    '''Calculate trading charges.'''
    df = pd.DataFrame(index=date, columns=['date_shift', 'charges'])
    df.date_shift = df.index.values
    df.date_shift = df.date_shift.shift()
    df.charges = np.where(df.index == df.date_shift, 0, commission)
    return df.charges

def donchian(prices, period):
    '''Calculate upper, lower, & middle Donchian lines.'''
    df = pd.DataFrame()
    df['upr'] = prices.high.rolling(period).max().shift(periods=1)
    df['lwr'] = prices.low.rolling(period).min().shift(periods=1)
    df['mid'] = 0.5 * (df.upr + df.lwr)
    return df

def shares_div4(shares):
    '''Modify number of shares to be purchased to be divisible by 4.'''
    s = pd.Series(shares.values)
    for i in s.index:
        while s.iloc[i] % 4 != 0:
            s.iloc[i] += 1
    return shares

def state(entry_signal, exit_signal, period):
    '''Calculate trade state signals.'''
    df = pd.concat([entry_signal, exit_signal], axis=1)
    df.columns = ['entry', 'exit']
    df['state'] = 0
    for i in range(period, len(df)):
        if df.loc[df.index[i], 'entry'] == 1 \
        and df.loc[df.index[i - 1], 'state'] == 0:
            df.loc[df.index[i], 'state'] = 1
        elif df.loc[df.index[i], 'exit'] == 1:
            df.loc[df.index[i], 'state'] = 0
        else:
            df.loc[df.index[i], 'state'] = df.loc[df.index[i - 1], 'state']
    return df.state

def trade_summary(trade_list):
    '''Generate trade summary.'''
    frame = pd.DataFrame(columns=['trade', 'entry', 'volatility','cost', 'exit', 'days', 'profit', 'pct', 'annual'])
    
    if trade_list.entry.iloc[-1] == 1:
        trade_num = trade_list.index.max()
    else:
        trade_num = trade_list.index.max() + 1
        
    for i in list(range(1, trade_num)):
        df = trade_list.loc[i]
        trade = df.index[0]
        entry = df.date.iloc[0]
        volatility = df.volatility.iloc[0]
        cost = df.cost.iloc[0]
        exit = df.date.iloc[-1]
        days = df.days.iloc[-1]
        profit = df.profit.sum()
        pct = (profit / cost) * 100
        annual = ((1 + pct / 100) ** (365 / days) - 1) * 100
        frame.loc[i] = [trade, entry, volatility, cost, exit, days, profit, pct, annual]
    frame = frame.set_index('trade')
    return frame

pd.set_option("display.max_columns", None)

In [4]:
# Import weekly closing prices.
prices = gta_prices.weekly(exchange, tidm)

In [5]:
# Donchian channels.
dc1 = donchian(prices, p1)
dc2 = donchian(prices, p2)
dc3 = donchian(prices, p3)
dc4 = donchian(prices, p4)

In [6]:
# System 1 entries & exits.
s1 = pd.concat([prices, dc1], axis=1)
s1['sys'] = 1
s1['buy'] = np.where(s1.close > s1.upr, 1, 0)
s1['sell'] = np.where(s1.close < s1.mid, 1, 0)
s1['state'] = state(s1.buy, s1.sell, p1)
s1['entry'] = np.where(np.logical_and(s1.state == 1, s1.state.shift(periods=1) == 0), 1, 0)
s1['exit'] = np.where(np.logical_and(s1.state == 0, s1.state.shift(periods=1) == 1), 1, 0)

In [7]:
# System 2 entries & exits.
s2 = pd.concat([prices, dc2], axis=1)
s2['sys'] = 2
s2['buy'] = s1.entry
s2['sell'] = np.where(s2.close < s2.mid, 1, 0)
s2['state'] = state(s2.buy, s2.sell, p2)
s2['entry'] = np.where(np.logical_and(s2.state == 1, s2.state.shift(periods=1) == 0), 1, 0)
s2['exit'] = np.where(np.logical_and(s2.state == 0, s2.state.shift(periods=1) == 1), 1, 0)

In [8]:
# System 3 entries & exits.
s3 = pd.concat([prices, dc3], axis=1)
s3['sys'] = 3
s3['buy'] = s1.entry
s3['sell'] = np.where(s3.close < s3.mid, 1, 0)
s3['state'] = state(s3.buy, s3.sell, p3)
s3['entry'] = np.where(np.logical_and(s3.state == 1, s3.state.shift(periods=1) == 0), 1, 0)
s3['exit'] = np.where(np.logical_and(s3.state == 0, s3.state.shift(periods=1) == 1), 1, 0)

In [9]:
# System 4 entries & exits.
s4 = pd.concat([prices, dc4], axis=1)
s4['sys'] = 4
s4['buy'] = s1.entry
s4['sell'] = np.where(s4.close < s4.mid, 1, 0)
s4['state'] = state(s4.buy, s4.sell, p4)
s4['entry'] = np.where(np.logical_and(s4.state == 1, s4.state.shift(periods=1) == 0), 1, 0)
s4['exit'] = np.where(np.logical_and(s4.state == 0, s4.state.shift(periods=1) == 1), 1, 0)

In [10]:
# Trade list indexed by date.
td = pd.concat([s1[s1.entry == 1] , s1[s1.exit == 1], s2[s2.exit == 1], s3[s3.exit == 1], s4[s4.exit == 1]], axis=0)
td = td.sort_index()

In [11]:
# Position size (buy).
td['volatility'] = np.where(td.entry == 1, abs((td.mid - td.close) / td.close).round(3), 0)
td['risk_amt'] = np.where(td.entry == 1, ((position_size * risk_pct) / td.volatility).round(2), 0)
td['shares'] = np.where(td.entry == 1, (td.risk_amt / td.close).astype('int'), 0)
td.shares = shares_div4(td.shares) # Modify number of shares to be purchased to be divisible by 4.
td.risk_amt = np.where(td.entry == 1, (td.close * td.shares).round(2), 0) # Adjust risk amount for revised share count.

In [12]:
# Position size (sell).
for index, row in td.iterrows():
    if row['entry'] == 1:
        shares = row['shares']
    elif row['exit'] == 1:
        td.at[index, 'shares'] = int(shares / 4)

In [13]:
# Charges.
td['charges'] = charges(td.index, commission)

In [14]:
# Stamp duty.
td['sduty'] = np.where(td.entry==1, ((sduty / 100) * td.close * td.shares).round(2), 0)

In [15]:
# Cost (buy).
td['cost'] = np.where(td.entry == 1, ((td.close * td.shares) + td.charges + td.sduty).round(2), 0)

In [16]:
# Cost (sell).
for index, row in td.iterrows():
    if row['entry'] == 1:
        cost = row['cost']
    elif row['exit'] == 1:
        td.at[index, 'cost'] = (cost / 4).round(2)

In [17]:
# Value (sell).
td['value'] = np.where(td.exit == 1, ((td.close * td.shares) - td.charges).round(2), 0)

In [18]:
# Profit.
td['profit'] = np.where(td.exit == 1, td.value - td.cost, 0)

In [19]:
# Cumulative profit.
td['cum_profit'] = td.profit.cumsum()

In [20]:
# Percentage return.
td['pct'] = np.where(td.exit == 1, ((td.profit / td.cost) * 100).round(1), 0)

In [21]:
# Trade duration.
td['days'] = 0
for index, row in td.iterrows():
    if row['entry'] == 1:
        start_date = index
    elif row['exit'] == 1:
        td.at[index, 'days'] = index - start_date
td.days = td.days.astype('timedelta64[D]')
td.days = td.days.dt.days

In [22]:
# Annual percentage return.
td['annual'] = ((np.power(1 + td.profit / td.cost, (365 / td.days)) - 1) * 100).round(1)

In [23]:
# Trade list indexed by trade.
td['trade'] = td.state.cumsum()
td = td.reset_index()
td = td.set_index('trade')
td

,date,open,high,low,close,upr,lwr,mid,sys,buy,sell,state,entry,exit,volatility,risk_amt,shares,charges,sduty,cost,value,profit,cum_profit,pct,days,annual
trade,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2001-05-18,0.57550,0.58951,0.57550,0.58864,0.57900,0.42483,0.501915,1,1,0,1,1,0,0.147,10204.66,17336,11.95,51.02,10267.63,0.00,0.00,0.00,0.0,0,0.0
1,2001-06-29,0.60615,0.60615,0.58951,0.58951,0.62280,0.55710,0.589950,4,0,1,0,0,1,0.000,0.00,4334,11.95,0.00,2566.91,2542.99,-23.92,-23.92,-0.9,42,-7.8
1,2001-07-06,0.58951,0.58951,0.56323,0.56323,0.62280,0.50717,0.564985,3,0,1,0,0,1,0.000,0.00,4334,11.95,0.00,2566.91,2429.09,-137.82,-161.74,-5.4,49,-33.7
1,2001-07-06,0.58951,0.58951,0.56323,0.56323,0.62280,0.50542,0.564110,2,0,1,0,0,1,0.000,0.00,4334,0.00,0.00,2566.91,2441.04,-125.87,-287.61,-4.9,49,-31.2
1,2002-02-22,0.61316,0.61579,0.59389,0.59564,0.68937,0.50542,0.597395,1,0,1,0,0,1,0.000,0.00,4334,11.95,0.00,2566.91,2569.55,2.64,-284.97,0.1,280,0.1
2,2003-06-20,0.55097,0.57024,0.54747,0.56937,0.56323,0.36089,0.462060,1,1,0,1,1,0,0.188,7980.29,14016,11.95,39.90,8032.14,0.00,0.00,-284.97,0.0,0,0.0
2,2003-10-03,0.72966,0.73142,0.71039,0.72090,0.75945,0.70076,0.730105,4,0,1,0,0,1,0.000,0.00,3504,11.95,0.00,2008.04,2514.08,506.04,221.07,25.2,105,118.4
2,2003-10-17,0.70514,0.71915,0.70514,0.71565,0.75945,0.67711,0.718280,3,0,1,0,0,1,0.000,0.00,3504,11.95,0.00,2008.04,2495.69,487.65,708.72,24.3,119,94.8
2,2003-12-19,0.67973,0.67973,0.65521,0.65521,0.75945,0.58951,0.674480,2,0,1,0,0,1,0.000,0.00,3504,11.95,0.00,2008.04,2283.91,275.87,984.59,13.7,182,29.5


In [24]:
# Trade summary.
tds = trade_summary(td)
tds.round({'volatility': 3, 'pct': 1, 'annual': 1})

,entry,volatility,cost,exit,days,profit,pct,annual
trade,,,,,,,,
1,2001-05-18,0.147,10267.63,2002-02-22,280,-284.97,-2.8,-3.6
2,2003-06-20,0.188,8032.14,2004-08-27,434,1723.43,21.5,17.8
3,2005-02-18,0.109,13842.25,2007-09-28,952,12925.27,93.4,28.8
4,2007-11-02,0.171,8830.72,2008-01-11,70,-993.05,-11.2,-46.3
5,2008-02-01,0.136,11105.52,2008-09-12,224,167.74,1.5,2.5
6,2009-09-18,0.194,7784.52,2010-08-06,322,1783.11,22.9,26.3
7,2016-11-25,0.234,6454.88,2017-10-13,322,604.62,9.4,10.7
8,2018-08-10,0.160,9432.74,2018-08-17,7,-1580.11,-16.8,-100.0
9,2019-09-20,0.172,8781.66,2020-03-13,175,61.68,0.7,1.5
